In [4]:
import json
import pandas as pd

# 1. NL to actions

In [5]:
# read all train tasks json data 
cnt = []
for i in range(1,7):
    df = pd.read_json("tw_alfred_seq2seq_train_task"+str(i)+"_hc.json")
    print(len(df))
    cnt.extend(df.data.to_list())
len(cnt)

790
308
650
459
533
813


3553

In [67]:
# cnt[0]["task"], cnt[0]["steps"]

We want to get top 10 types of tasks

In [6]:
# get total types of tasks based on first word
tasks = [x["task"] for x in cnt]
steps = [x["steps"] for x in cnt]
len(tasks), len(steps), set([x.split()[0] for x in tasks])

(3553, 3553, {'clean', 'cool', 'examine', 'find', 'heat', 'look', 'put'})

In [14]:
# tasks[1:10]

['put a pillow in sofa.',
 'put some spraybottle on cabinet.',
 'put a peppershaker in sidetable.',
 'put some toiletpaper on toiletpaperhanger.',
 'put some keychain on sofa.',
 'put some spraybottle on countertop.',
 'put some alarmclock on dresser.',
 'put some cellphone on sidetable.',
 'put some laptop on desk.']

Looks like we also have complex tasks too. How many?

In [8]:
len([x for x in tasks if " and " in x])/ len(tasks)

0.35040810582606247

In [9]:
len([x for x in tasks if " in " in x])/ len(tasks)

0.794821277793414

In [10]:
len([x for x in tasks if " on " in x])/ len(tasks)

0.118491415705038

any other prominent stopwords? - TODO

Let's look at each task type

In [11]:
put_tasks = [(idx, x) for idx, x in enumerate(tasks) if "put" in x and " and " not in x]
clean_tasks = [(idx, x) for idx, x in enumerate(tasks) if "clean" in x ]
heat_tasks = [(idx, x) for idx, x in enumerate(tasks) if "heat" in x ]
cool_tasks = [(idx, x) for idx, x in enumerate(tasks) if "cool" in x ]
examine_tasks = [(idx, x) for idx, x in enumerate(tasks) if "examine" in x ]
find_tasks = [(idx, x) for idx, x in enumerate(tasks) if "find" in x ]
look_tasks = [(idx, x) for idx, x in enumerate(tasks) if "look" in x ]

len(put_tasks), len(clean_tasks), len(heat_tasks), len(cool_tasks), len(examine_tasks), len(find_tasks), len(look_tasks)

(2000, 650, 220, 533, 176, 433, 132)

In [17]:
# heat_tasks

<!-- 9th gen i7 8 cores - 13gb cpu ram -->

In [18]:
idxs = [794, 1088, 1124, 1195, 1555, 1787, 2228, 2326, 2816, 2829]

In [19]:
for idx in idxs:
    print(tasks[idx])

examine the alarmclock with the desklamp.
look at book under the desklamp.
put a clean bowl in fridge.
clean some mug and put it in coffeemachine.
put a clean knife in drawer.
heat some plate and put it in countertop.
put a cool plate in diningtable.
cool some winebottle and put it in diningtable.
find two newspaper and put them in armchair.
find two remotecontrol and put them in armchair.


In [162]:
# idxs = [794, 1088, 1124, 1195, 1555, 1787, 2228, 2326, 2816, 2829]
with open("selected_nl-action_pairs.csv", "w") as f:
    for idx in idxs:
        f.write(str(idx) +","+ tasks[idx] + ',"')
        actions = [x["action"] for x in steps[idx]]
        for idx, action in enumerate(actions):
            if idx < len(actions)-1:
                f.write( action + ",")
            else:
                f.write(action)
        f.write('"\n')


In [62]:
idx = 0
cnt[idx]["steps"][0]

{'step_id': 0,
 'obs': '-= Welcome to TextWorld, ALFRED! =- You are in the middle of a room. Looking quickly around you, you see a countertop 1, a drawer 1, a drawer 2, a drawer 3, a drawer 4, a drawer 5, a drawer 6, a garbagecan 1, a handtowelholder 1, a handtowelholder 2, a sinkbasin 1, a toilet 1, a toiletpaperhanger 1, a towelholder 1, and a towelholder 2.',
 'action': 'look'}

In [60]:
idx = 0
actions = [step["action"] for step in cnt[idx]["steps"]]
# actions

In [61]:
idx = 0
observations = [step["obs"] for step in cnt[idx]["steps"]]
# observations 

# 2. NL to predicates

In [22]:
pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp from https://files.pythonhosted.org/packages/fd/2f/d203acfe3a2d1e2823fef5876ff342cc34d08c57748baac2fc67914cea22/aiohttp-3.8.6-cp310-cp310-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for async-timeout<5.0,>=4.0.0a3 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 664.0 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for frozenlist>=1.1.1 from https://files.pythonhosted.org/packages/a3/5b/c785feda30d9fda8c1b1a11941e91253f59aeaf13e87ebe908d0f3f6c628/frozenlist-1.4.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
import os
import codecs
import openai
openai.organization = "org-7k2wTcWtQchVCGMLHbLh9BDo"
openai.api_key = os.getenv('OPENAI_API_KEY')

In [27]:
models = openai.Model.list().data
model_ids = [x.id for x in models]
# model_ids

In [28]:
tasks_gpt3 = []
for idx in idxs:
    prompt = "What is the first-order logic predicate formulation \
    for the following natural language task '" + tasks[idx] + "'?" 
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    tasks_gpt3.append(completion.choices[0].message)

In [57]:
print(tasks_gpt3[0].content)

Let A(x) represent the predicate "x is an alarm clock" and L(x) represent the predicate "x is a desk lamp."

The first-order logic predicate formulation for the given natural language task formulation would be:

∃x (A(x) ∧ L(x))

This translates to "There exists an object x such that x is an alarm clock and x is a desk lamp."


In [55]:
[x.content for x in tasks_gpt3]

['Let A(x) represent the predicate "x is an alarm clock" and L(x) represent the predicate "x is a desk lamp."\n\nThe first-order logic predicate formulation for the given natural language task formulation would be:\n\n∃x (A(x) ∧ L(x))\n\nThis translates to "There exists an object x such that x is an alarm clock and x is a desk lamp."',
 'Let L(x) represent "x is a lamp," B(x) represent "x is a book," U(x, y) represent "x is under y," and G(x, y) represent "x is looking at y." \n\nThe first-order logic predicate formulation for the given task formulation would be:\n\n∃x∃y (B(x) ∧ L(y) ∧ U(x, y) ∧ G(x, y))',
 'Let C(x) represent the predicate "x is a bowl" and F(x) represent the predicate "x is in the fridge". Additionally, let B(x) represent the predicate "x is clean".\n\nThe first-order logic predicate formulation for the task "put a clean bowl in fridge" can be represented as follows:\n\n∃x [C(x) ∧ B(x) ∧ F(x)]\n\nThis statement can be read as "There exists an x such that x is a bowl,

In [54]:
# [x.content for x in tasks_gpt3]

In [31]:
tasks_gpt3v2 = []
for idx in idxs:
    prompt = "What is the first-order logic predicate formulation for the following natural language task '" + tasks[idx] + "'?" 
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    tasks_gpt3.append(completion.choices[0].message)

In [38]:
responses = [x.content for x in tasks_gpt3v2]


In [46]:
tasks_gpt3_1shotv2 = []
for idx in idxs:
    prompt = "The first-order logic predicate representation of \
    'put a clean knife in drawer' is 'put(knife, in(knife, drawer)), clean(knife)'.\
    The first-order logic predicate representation of '" + tasks[idx] + "' is " 
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    tasks_gpt3_1shotv2.append(completion.choices[0].message)

In [51]:
for idx in idxs:
    print(tasks[idx])

examine the alarmclock with the desklamp.
look at book under the desklamp.
put a clean bowl in fridge.
clean some mug and put it in coffeemachine.
put a clean knife in drawer.
heat some plate and put it in countertop.
put a cool plate in diningtable.
cool some winebottle and put it in diningtable.
find two newspaper and put them in armchair.
find two remotecontrol and put them in armchair.


In [48]:
# prompt, 
[x.content for x in tasks_gpt3_1shotv2]

['examine(alarm_clock, with(alarm_clock, desklamp)), true(alarm_clock), true(desklamp)',
 'look(at(book, under(desklamp))).',
 "'put(bowl, in(bowl, fridge)), clean(bowl)'.",
 'clean(mug) and put(mug, in(mug, coffeemachine)).',
 "'put(knife, in(knife, drawer)), clean(knife)'",
 'put(plate, in(plate, countertop)), heat(plate)',
 "'put(plate, in(plate, diningtable)), cool(plate)'.",
 'cool(winebottle), put(winebottle, in(winebottle, diningtable)).',
 'put(newspaper1, in(newspaper1, armchair)), put(newspaper2, in(newspaper2, armchair)), find(newspaper1), find(newspaper2)',
 'find(remotecontrol, and(find(remotecontrol, put(remotecontrol, in(armchair)))), remotecontrol).\n']

In [50]:
print(prompt)

The first-order logic predicate representation of 'put a clean knife in drawer' is 'put(knife, in(knife, drawer)), clean(knife)'. The first-order logic predicate representation of 'find two remotecontrol and put them in armchair.' is 


In [45]:
prompt, [x.content for x in tasks_gpt3_1shotv2]

("You know that the first-order logic predicate representation of 'put a clean knife in drawer.' is 'put(knife, in(knife, drawer)), clean(knife)'. Write the first-order logic predicate representation for the following natural language task 'find two remotecontrol and put them in armchair.'.",
 ['examine(alarmclock, with(alarmclock, desklamp))',
  'look(at(book, under(desklamp)))',
  'The first-order logic predicate representation for the task "put a clean bowl in fridge" can be represented as follows: \n\nput(bowl, in(bowl, fridge)), clean(bowl)',
  'clean(mug) and put(mug, in(mug, coffeemachine)).',
  'put(knife, in(drawer)), clean(knife)',
  'The first-order logic predicate representation for the given task would be:\n\nheat(plate) ∧ put(plate, in(plate, countertop))',
  "The first-order logic predicate representation for the given natural language task 'put a cool plate in diningtable' can be expressed as follows:\n\nput(plate, in(plate, diningtable)), cool(plate)",
  'cool(winebott

In [42]:
[x.content for x in tasks_gpt3_1shot]

['examine(alarmclock, with(alarmclock, desklamp))',
 'look(at(book, under(desklamp)))',
 'The first-order logic predicate representation for the task "put a clean bowl in the fridge" would be:\n\nput(bowl, in(bowl, fridge)), clean(bowl)',
 'To represent the task "clean some mug and put it in coffeemachine" using first-order logic predicates, we can use the following representation:\n\n∃x (mug(x) ∧ clean(x) ∧ put(x, in(x, coffeemachine)))\n\nExplanation:\n\n∃x - There exists an object x.\nmug(x) - x is a mug.\nclean(x) - x is clean.\nput(x, in(x, coffeemachine)) - x is put inside the coffeemachine.',
 'put(knife, in(drawer)), clean(knife)',
 'heat(plate) ∧ put(plate, in(plate, countertop))',
 'To represent the task "put a cool plate in dining table" in first-order logic predicate, we can use the following representation:\n\nput(plate, in(plate, dining_table)), cool(plate)\n\nWhere:\n- put(plate, in(plate, dining_table)) represents the action of putting the plate in the dining table.\n- 

In [ ]:
tasks_gpt3_1shotv2 = []
for idx in idxs:
    prompt = "The first-order logic predicate representation of 'examine alarm clock with desklamp' is 'examine(alarmclock, in(knife, drawer)), clean(knife)'. The first-order logic predicate representation of '" + tasks[idx] + "' is " 
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    tasks_gpt3_1shotv2.append(completion.choices[0].message)